## Heat transfer using Physics-Informed Neural Network

In [ ]:
import sys
sys.path.append('./src')

In [ ]:
from heat_solver import HeatForwardSolver
import plotter
from params import u0, options, boundaries
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
from pinn import PINN
import data_generation as dg
from IPython.display import HTML

### Numerical simulation

In [ ]:
solver = HeatForwardSolver(**options)
solver.set_initial(u0)
solver.set_boundaries(boundaries)

solver.solve()

plt = plotter.plot_frame(solver.u, solver.u.shape[2]-1, solver.delta_t)

anim = plotter.animate_plot(solver.u, solver.delta_t, filename='numerical_prediction.gif')

plt.show()
# HTML(anim.to_html5_video())

### PINN traning and prediciton

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

pinn = PINN([3, 40, 40, 40, 40, 40, 1], device)

traning_data = dg.generate_traning_data(device, n_data=50_000, n_colloc=10_000, n_bc=1000, n_ic=400)

In [ ]:
pinn.train()
pinn.start_train(traning_data, 20_000)

n = options['domain_length']
iter_end = options['max_iter']
u = np.zeros((n, n, iter_end))
pinn.eval()

for i in tqdm(range(iter_end), desc='Predicting'):
	temp = np.linspace(0, n, n) / n
	X0, Y0 = np.meshgrid(temp, temp)

	X = X0.reshape([n*n, 1])
	Y = Y0.reshape([n*n, 1])

	t = (i * options['delta_t']) / options['t_end']
	grid_points = np.concatenate(
		(X, Y,
		np.full((n*n, 1), t)),
		axis=1)

	grid_points = torch.tensor(grid_points, dtype=torch.float32).to(device)

	u_pred = pinn.forward(grid_points)
	u_pred = u_pred.cpu().detach().numpy().reshape((n, n))

	u[:, :, i] = u_pred * 100

plotter.plot_frame(u, iter_end-1, options['delta_t']).show()
plotter.animate_plot(u, options['delta_t'], filename="pinn_prediction.gif")